# Feature Engineering

In [3]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import plotly.express as px
from IPython.display import display
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from yellowbrick.model_selection import RFECV
import warnings
warnings.filterwarnings("ignore")
from math import sqrt 
from sklearn import metrics
import os
import pastry

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE 


import warnings
warnings.filterwarnings("ignore")


data = pd.read_csv('data/WA_Fn-UseC_-HR-Employee-Attrition.csv')
data['attrition_binary'] = pd.Series(np.where(data.Attrition.values == 'Yes', 1, 0),data.index)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [4]:
cate_var = [list(data.columns)[i] for i in [2,4,6,7,10,11,13,14,15,16,17,21,22,25,27,30]] #store all categorical features' name in a list
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['Attrition','attrition_binary','EmployeeNumber','Education','Gender','Over18']),data['attrition_binary'], test_size=0.25, random_state=42)
cate_var_index_data = [1,3,5,7,9,10,11,12,13,17,19,20,22,25] # list containing indexes of all categorical variables in X_train
cate_var = [list(data.drop(columns=['Attrition','attrition_binary','EmployeeNumber','Education','Gender','Over18']).columns)[i] for i in cate_var_index_data] # dropping statistically insignificant features
sm = SMOTENC(sampling_strategy=1.0,random_state=42, categorical_features = cate_var_index_data,k_neighbors=5 )
X_res, y_res = sm.fit_resample(X_train, y_train) #fitting SMOTE on training set

encoding_drop_col_list = [] # define the list containing the levels to be dropped after one hot encoding
for i in cate_var:
    temp_df = pd.get_dummies(X_train[i], prefix=i)
    encoding_drop_col_list.append(list(temp_df.columns)[0])

for i in cate_var:  # one hot encoding for training set without SMOTE
    temp_df = pd.get_dummies(X_res[i], prefix=i)
    X_res_temp =X_res.join(temp_df)
    X_res = X_res_temp
X_res.drop(columns = cate_var+encoding_drop_col_list,inplace=True)

for i in cate_var: # one hot encoding for training set with SMOTE
    temp_df = pd.get_dummies(X_test[i], prefix=i)
    X_test_temp =X_test.join(temp_df)
    X_test = X_test_temp
X_test.drop(columns = cate_var+encoding_drop_col_list,inplace=True)

In [5]:
# find all first order interactions
def find_second_order_dependency(df:pd.DataFrame(),cate_var:list)->list:
    dependency_list = []
    for i in range(len(cate_var)):
        for j in range(i+1,len(cate_var)):
            i_col = cate_var[i]
            j_col = cate_var[j]
            chi2_result = stats.chi2_contingency(pd.DataFrame.to_numpy(pd.pivot_table(df, values='attrition_binary', index=[i_col],columns=[j_col], aggfunc=np.count_nonzero)))
            if chi2_result[1]<0.05:
                dependency_list.append([i_col,j_col])
    return dependency_list
find_second_order_dependency(data,cate_var)

[['BusinessTravel', 'JobInvolvement'],
 ['Department', 'JobLevel'],
 ['Department', 'MaritalStatus'],
 ['EducationField', 'MaritalStatus'],
 ['EnvironmentSatisfaction', 'JobInvolvement'],
 ['JobInvolvement', 'JobLevel'],
 ['JobRole', 'MaritalStatus'],
 ['JobRole', 'RelationshipSatisfaction']]

In [6]:
# feature engineering by adding interaction terms
X_res_interaction = X_res.copy()
X_test_interaction = X_test.copy()
for i in range(len(find_second_order_dependency(data,cate_var))):
    col1 = find_second_order_dependency(data,cate_var)[i][0]
    col2 = find_second_order_dependency(data,cate_var)[i][1]
    col1_levels = [j for j in list(X_res.columns) if col1 in j]
    col2_levels = [k for k in list(X_res.columns) if col2 in k]
    col1_list = []
    col2_list = []
    for m in col1_levels:
        col1_list = [p for p in X_res[m].to_list()]
        col1_list_test = [p for p in X_test[m].to_list()]
        for n in col2_levels:
            col2_list =[q for q in X_res[n].to_list()]
            col2_list_test =[q for q in X_test[n].to_list()]
            multiplied_list = [col1_list[i]*col2_list[i] for i in range(len(col1_list))]
            multiplied_list_test = [col1_list_test[i]*col2_list_test[i] for i in range(len(col1_list_test))]
            temp_merged_name = m+'_'+n
            X_res_interaction[temp_merged_name] = multiplied_list
            X_test_interaction[temp_merged_name] = multiplied_list_test
X_res = X_res_interaction
X_test = X_test_interaction
# output dataframes: X_res, y_res, X_test, y_test

## Data Shape

In [13]:
print('Number of features in original data: ', len(data.columns))
print('\nNumber of features in new Data: ', len(X_res.columns))

Number of features in original data:  36

Number of features in new Data:  148


In [17]:
m=Classification()

In [24]:
type(m.X_train)

numpy.ndarray

In [26]:
type(X_res.values)

numpy.ndarray

# ANN

In [27]:
def buildANNModel(data):
    cl = Sequential()
    cl.add(Dense(20, activation='relu', kernel_initializer='random_normal',  input_shape=(data.shape[1],)))
    cl.add(Dropout(0.1))
    cl.add(Dense(10, activation='relu', kernel_initializer='random_normal',  input_shape=(data.shape[1],)))
    cl.add(Dropout(0.1))
    cl.add(Dense(5, activation='relu', kernel_initializer='random_normal',  input_shape=(data.shape[1],)))
    cl.add(Dropout(0.1))
    cl.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
    adm = Adam()
    cl.compile(optimizer=adm, loss='binary_crossentropy', metrics=['accuracy'])
    return cl

def trainANNModel(X_train, y_train, epochs=10, batch_size=10, validation_split=0.33, verbose=0):
    #model = buildANNModel(X_train)
    model.fit(X_train, y_train, validation_split=validation_split, epochs=epochs, batch_size=batch_size, verbose=verbose)

def evaluateANNModel(model, testX,testy):
    yhat_probs = model.predict(testX, verbose=0)
    yhat_classes = model.predict_classes(testX, verbose=0)
    yhat_probs = yhat_probs[:, 0]
    yhat_classes = yhat_classes[:, 0]

    accuracy = accuracy_score(testy, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testy, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testy, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testy, yhat_classes)
    print('F1 score: %f' % f1)

In [32]:
model = buildANNModel(X_res)
trainANNModel(X_res,y_res)
evaluateANNModel(model, X_test,y_test)

#ann.trainANNModel(X_res,y_res)
#ann.evaluateANNModel(X_test,y_test)

Accuracy: 0.130435
Precision: 0.130435
Recall: 1.000000
F1 score: 0.230769


# Support Vector Machine

In [33]:
model = Classification()
data = model.data

from sklearn import svm
from sklearn.metrics import auc,classification_report, roc_curve, roc_auc_score

clf = svm.SVC()
clf.fit(X_res, y_res)
predicted = clf.predict(X_test)

print('Kernel Degree = 3')
# get the accuracy
print('Accuracy: ',accuracy_score(y_test, predicted))
print('ROC AUC Score: ', roc_auc_score(y_test, predicted))

Kernel Degree = 3
Accuracy:  0.5679347826086957
ROC AUC Score:  0.5479166666666667


In [7]:
from sklearn import svm
from sklearn.metrics import auc,classification_report, roc_curve, roc_auc_score

clf = svm.SVC(kernel='linear', max_iter=100)
clf.fit(X_res, y_res)
predicted = clf.predict(X_test)

print('Kernel Degree = 3')
# get the accuracy
print('Accuracy: ',accuracy_score(y_test, predicted))
print('ROC AUC Score: ', roc_auc_score(y_test, predicted))

accuracy = accuracy_score(testy, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(testy, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(testy, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(testy, yhat_classes)
print('F1 score: %f' % f1)

Kernel Degree = 3
Accuracy:  0.42391304347826086
ROC AUC Score:  0.509375
